**To do:**
    
* Get rid of logging
* Add better indentation for pretty printing

In [59]:
# To do:


import inspect
#import logging #Do I really wanna use logging for this? Logging is actually turning into sort of a PITA
from collections import Counter
import re
import warnings
import pprint
import sys

# Again... do I really want to use logging here? 
# It's a simple way to get the timestamp and I like that the messages show as colored (in jupyter), 
# but otherwise it's sort of a pain. Does all sorts of weird shit with white space.

#logging.basicConfig(
#    format="[%(asctime)s] %(message)s",
#    level=logging.INFO,
#    stream=sys.stdout
#)

class DebugHelper(object):
    def __init__(self, parent_logger_name='DebugHelper'):
        self.scope_calls = Counter()
        #self.logger = logging.getLogger(parent_logger_name)
        #self.logger.setLevel(logging.INFO)
        
        init_call = self._calling_context(2)
        self._init_name = re.findall('(\w+)\s?=\s?DebugHelper\\(', init_call)[0].strip()
    def _pformat(self, v):
        """pretty print formatting for multi-line outputs"""
        v = pprint.pformat(v)
        if '\n' in v:
            v = '\n' + v
        return v
    @staticmethod
    def _ts():
        return str(dt.datetime.now())
    def _msg(self, msg_str):
        ts = self._ts()
        #bgnd = "\u001b[45m"
        code = 223 # 224 is the same color jupyter uses for stderr.
        bgnd = u"\u001b[48;5;" + str(code) + "m "
        reset = "\u001b[0m"
        print("{}[{}] {} {}".format(bgnd, ts, msg_str, reset))
        #self.logger.info(msg)
    def msg(self, *args, **kwargs):
        """Workhorse"""
        scope = self._calling_scope()
        self.scope_calls[scope] += 1
        n = self.scope_calls[scope]
        msg_str = '[{}] {}'.format(scope, n)
        #self.logger.info(msg_str)
        self._msg(msg_str)
        
        parts = []
        if args:
            parts.extend(self._make_argstr(args))
        if kwargs:
            for k,v in kwargs.items():
                parts.append('... {}: {}'.format(k, self._pformat(v)))
        for i, msg_str in enumerate(parts):
            if i == (len(parts)-1):
                msg_str += '\n'
            #self.logger.info(msg_str)
            self._msg(msg_str)
        #print()
    def _calling_context(self, i):
        context = inspect.stack()[i].code_context[0].strip()
        # Handle comments
        if '#' in context:
            context, *_ = context.split('#')
        return context
    def _calling_scope(self):
        scope = self._calling_context(4)
        if scope == 'exec(code_obj, self.user_global_ns, self.user_ns)':
            scope = '__main__'
        return scope
    def _msg_argnames(self):
        src_call = self._calling_context(4)
        call_pat = '{}.msg('.format(self._init_name)
        _, args1 = src_call.split(call_pat)
        args2 = args1.strip()[:-1] # assumes no comments on line
        args = args2.split(',')
        argnames = []
        for a in args:
            if a:
                if (a[0]==a[-1]) and (a[0] in ('"', "'")):
                    argnames.append(None)
                elif '=' not in a:
                    argnames.append(a)
        return argnames
    def _make_argstr(self, args):
        argnames = self._msg_argnames()
        argstrs = []
        for argname, arg in zip(argnames, args):
            if argname:
                argstrs.append('...{}: {}'.format(argname, self._pformat(arg)))
            else:
                argstrs.append('...{}'.format(arg))
        argstrs[0] = '... ' + argstrs[0][3:]
        return argstrs
    
dbgr = DebugHelper()

class TestClass(object):
    def __init__(self, arg):
        dbgr.msg()
        dbgr.msg('top')
        dbgr.msg(arg)
        b = self.msg(arg)
        dbgr.msg(b)
        self.msg(arg+'bar')
        dbgr.msg()
    def msg(self, foo):
        dbgr.msg()
        dbgr.msg(foo)
        b = '~~{}~~'.format(foo)
        dbgr.msg(b)
        print(b)
        return b
    
TestClass('foo')

 [2018-03-20 02:32:54.166560] [TestClass('foo')] 1 
 [2018-03-20 02:32:54.179088] [TestClass('foo')] 2 
 [2018-03-20 02:32:54.193600] ... top
 
 [2018-03-20 02:32:54.205661] [TestClass('foo')] 3 
 [2018-03-20 02:32:54.220195] ... arg: 'foo'
 
 [2018-03-20 02:32:54.234238] [b = self.msg(arg)] 1 
 [2018-03-20 02:32:54.247763] [b = self.msg(arg)] 2 
 [2018-03-20 02:32:54.264312] ... foo: 'foo'
 
 [2018-03-20 02:32:54.278349] [b = self.msg(arg)] 3 
 [2018-03-20 02:32:54.293890] ... b: '~~foo~~'
 
~~foo~~
 [2018-03-20 02:32:54.305418] [TestClass('foo')] 4 
 [2018-03-20 02:32:54.321972] ... b: '~~foo~~'
 
 [2018-03-20 02:32:54.338515] [self.msg(arg+'bar')] 1 
 [2018-03-20 02:32:54.352553] [self.msg(arg+'bar')] 2 
 [2018-03-20 02:32:54.368596] ... foo: 'foobar'
 
 [2018-03-20 02:32:54.382601] [self.msg(arg+'bar')] 3 
 [2018-03-20 02:32:54.402184] ... b: '~~foobar~~'
 
~~foobar~~
 [2018-03-20 02:32:54.419224] [TestClass('foo')] 5 


In [1]:
from debugger import DebugHelper

In [1]:
import debugger
dbgr = debugger.DebugHelper()

class TestClass(object):
    def __init__(self, arg):
        dbgr.msg()
        dbgr.msg('top')
        dbgr.msg(arg)
        b = self.msg(arg)
        dbgr.msg(b)
        self.msg(arg+'bar')
        dbgr.msg()
    def msg(self, foo):
        dbgr.msg()
        dbgr.msg(foo)
        b = '~~{}~~'.format(foo)
        dbgr.msg(b)
        print(b)
        return b
    
TestClass('foo')

IndexError: list index out of range

In [33]:
logging.INFO

20

In [35]:
dbgr.logger.getEffectiveLevel()

30

In [40]:
warnings.warn('foobar')

C:\Users\shagg\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: UserWarning: foobar
  """Entry point for launching an IPython kernel.


In [43]:
# ANSI escape codes
#   http://www.lihaoyi.com/post/BuildyourownCommandLinewithANSIescapecodes.html
print("foo")
#print( u"\u001b[31m")
print("\u001b[45m")
print("bar")
print (u"\u001b[0m")
print("baz")

foo

bar

baz


In [56]:
import numpy as np

np.linspace

<function numpy.core.function_base.linspace>

In [46]:
from sklearn.linear_model import LinearRegression
import numpy as np

m, b = 20, 30
x  = np.arange(1000)
y1 = m*x + b

'2018-03-20 01:01:48.142176'